In [8]:
import random

In [230]:
SPEED_LIMIT              = 10 
NUM_LANES                = 4
TRAFFIC_DENSITY          = 0.15
MAX_ACCEL                = 2

LEFT_LANE_SPEED          = SPEED_LIMIT - 1
SPEED_DIFF_BETWEEN_LANES = 1

RIGHT_LANE_SPEED         = LEFT_LANE_SPEED - (NUM_LANES-1)*SPEED_DIFF_BETWEEN_LANES

LANE_SPEEDS = [RIGHT_LANE_SPEED + i*SPEED_DIFF_BETWEEN_LANES for i in range(NUM_LANES)]

DT = 1
NUM_UPDATES = 250

def main():
    road = Road( SPEED_LIMIT, TRAFFIC_DENSITY, LANE_SPEEDS)
    road.populate_traffic(LANE_SPEEDS, TRAFFIC_DENSITY)
    road.add_ego(NUM_LANES[-1], 0 )
    
    sim = Simulator(road)
    
    
    planner = BehaviorPlanner(road, MAX_ACCEL)
    trajectory_gen = TrajectoryGenerator()
    
    for _ in range(NUM_UPDATES):
        # fetch data
        predictions = sim.get_predictions()
        localization = sim.get_localization()
        
        # feed data to planner
        planner.feed_localization(localization)
        planner.feed_predictions(predictions)
        
        # get new plan
        plan = planner.plan()
        
        # implement trajectory from plan
        trajectory = trajectory_gen.generate_trajectory(plan, localization, predictions)
        
        # in this case a trajectory is just a lane and an acceleration
        new_lane, new_a = trajectory
        
        # check that they are valid
        if not is_valid_lane_change(new_lane, ego.lane): raise ValueError
        if not is_valid_acceleration(new_a, ego.a, ego.v ): raise ValueError
            
        # update ego vehicle's state
        sim.set_ego_state(new_lane, new_a)
        
        # update all vehicles
        sim.update_vehicles()
        
        
        
        

In [231]:
class BehaviorPlanner(object):
    def __init__(self, num_lanes, speed_limit, max_accel):
        self.num_lanes = num_lanes
        self.speed_limit = speed_limit
        self.max_accel = max_accel
        pass
    
    def plan(self):
        pass
    
    def feed_localization(self, localization):
        pass
    
    def feed_predictions(predicitons):
        pass
    
class TrajectoryGenerator(object):
    def __init__(self):
        pass
    
    def generate_trajectory(self, plan, localization, predictions):
        pass

In [232]:
class Simulator(object):
    def __init__(self, lane_speeds, speed_limit):
        self.lane_speeds = lane_speeds
        self.speed_limit = speed_limit
    
    def get_predictions(self):
        pass
    
    def get_localization(self):
        pass
    
    def set_ego_state(self, new_lane, new_a):
        pass
    
    def update_vehicles(self):
        pass
    
    def __repr__(self):
        pass
     

In [233]:
   
def is_valid_lane_change(new_lane, old_lane):
    if new_lane >= NUM_LANES or new_lane < 0: return False
    if abs(new_lane-old_lane) > 1: return False
    return True

def is_valid_acceleration(new_a, old_a, old_v): 
    if new_a > MAX_ACCEL or new_a < -MAX_ACCEL: return False
    if new_a * DT + old_v > SPEED_LIMIT: return False
    return True


    

In [1232]:
class Road(object):
    update_width = 70
    ego_rep = " *** "
    ego_key = -1
    def __init__(self, speed_limit, traffic_density, lane_speeds):
        self.num_lanes = len(lane_speeds)
        self.lane_speeds = lane_speeds
        self.speed_limit = speed_limit
        self.density = traffic_density
        self.camera_center = self.update_width / 2
        self.vehicles = {}
        self.vehicles_added = 0
        
    def populate_traffic(self):
        start_s = max(self.camera_center - (self.update_width / 2), 0)
        for l in range(self.num_lanes):
            lane_speed = self.lane_speeds[l]
            vehicle_just_added = False
            for s in range(start_s, start_s + self.update_width):
                if vehicle_just_added:
                    vehicle_just_added = False
                    continue
                if random.random() < self.density:
                    vehicle = Vehicle(l, s, lane_speed, 0)
                    vehicle.state = "CS"
                    self.vehicles_added += 1
                    self.vehicles[self.vehicles_added] = vehicle
                    vehicle_just_added = True
                    
    def simulated_update(self):
        for _id, v in self.vehicles.items():
            is_ego = False
            if _id == self.ego_key:
                is_ego = True
            v.simulated_update(self.vehicles, is_ego)
        for v in self.vehicles.values():
            v.increment()
    
    def increment_vehicles(self):
        for v_id, veh in self.vehicles.items():
            if veh.state == "CS":
                veh.s = veh.s + veh.v
                veh.a = 0
            elif veh.state == "KL":
                in_front = [v for (v_id, v) in self.vehicles.items() if v.lane == veh.lane and v.s > veh.s ]
                if len(in_front) > 0:
                    leading = min(in_front, key=lambda v: v.s - veh.s)
                    delta_s = leading.s - veh.s
                    print "delta s between {} and {} is {}".format(v_id, self.ego_rep , delta_s)
                    if delta_s > 10:
                        print "setting a to 1"
                        veh.a = 1
                    elif leading.v < veh.v:
                        print "setting a to -1"
                        veh.a = -1
                    else: 
                        veh.a = 0
                
                
                veh.v = veh.v + veh.a
                veh.s = veh.s + veh.v 
#             elif v.state == "LCL":
#                 v.lane -= 1
#                 v.v = LANE_SPEEDS[v.lane]
#                 v.a = 0

#             elif self.state == "LCR"

    
    def add_ego(self, lane_num, s):
        for v_id, v in self.vehicles.items():
            if v.lane == lane_num and v.s == s:
                del self.vehicles[v_id]
        ego = Vehicle(lane_num, s, self.lane_speeds[lane_num], 0)
        ego.state = "KL"
        self.vehicles[self.ego_key] = ego
        
    def cull(self):
        ego = self.vehicles[self.ego_key]
        center_s = ego.s
        claimed = set([(v.lane, v.s) for v in self.vehicles.values()])
        for v_id, v in self.vehicles.items():
            if v.s > (center_s + self.update_width / 2) or v.s < (center_s - self.update_width / 2):
                try:
                    claimed.remove((v.lane,v.s))
                except:
                    continue
                del self.vehicles[v_id]
                
                placed = False
                while not placed:
                    lane_num = random.choice(range(self.num_lanes))
                    ds = random.choice(range(self.update_width/2-15,self.update_width/2 -1 ))
                    if lane_num > self.num_lanes / 2:
                        ds *= -1
                    s = center_s + ds
                    if (lane_num, s) not in claimed:
                        placed = True
                        speed = self.lane_speeds[lane_num]
                        vehicle = Vehicle(lane_num, s, speed, 0)
                        self.vehicles_added += 1
                        self.vehicles[self.vehicles_added] = vehicle
                        print 'adding vehicle {} at lane {} with s={}'.format(self.vehicles_added, lane_num, s)
        
    def __repr__(self):
        s = self.vehicles.get(self.ego_key).s
        self.camera_center = max(s, self.update_width / 2)
        s_min = max(self.camera_center - self.update_width /2, 0)
        s_max = s_min + self.update_width
        road = [["     " if i % 3 == 0 else "     "for ln in range(self.num_lanes)] for i in range(self.update_width)]
        for v_id, v in self.vehicles.items():
            if s_min <= v.s < s_max:
                if v_id == self.ego_key:
                    marker = self.ego_rep
                else:
                    marker = " %03d " % v_id
                road[v.s - s_min][v.lane] = marker
        s = ""
        for l in road:
            s += "|" + "|".join(l) + "|"
            s += "\n"
        return s
        

In [1233]:
class Vehicle(object):
    L = 1
    preferred_buffer = 12
    def __init__(self, lane, s, v, a):
        self.lane = lane
        self.s = s 
        self.v = v
        self.a = a
        self.preferred_speed = LANE_SPEEDS[lane]
        self.state = "CS"
        
    def simulated_update(self, vehicles, is_ego=False):
        in_front = [v for (v_id, v) in vehicles.items() if v.lane == self.lane and v.s > self.s ]
        
        F_net = 0
        
        dv = self.v - self.preferred_speed
        
        if is_ego:
            print "dv is: {} - {} = {}".format(self.v, self.preferred_speed, dv)
        F_v = -abs(dv) * float(dv) / 2
        if is_ego:
            print "F_v is {}".format(F_v)
        F_net += F_v

        if len(in_front) > 0:
            leading = min(in_front, key=lambda v: v.s - self.s)
            if is_ego:

                print "leading is s: {}".format(leading.s)
                print "leading v is: {}".format(leading.v)
            delta_s = leading.s - self.s
            if is_ego:
                print "delta s is {}".format(delta_s)
            if delta_s < self.preferred_buffer:
                F_back = -20 * (float(self.preferred_buffer - delta_s) / self.preferred_buffer)**2
                F_net += F_back
                if is_ego:
                    print "f back is {}".format(F_back)
            if delta_s > self.preferred_buffer:
                F_net += 1.0
            
            delta_v = leading.v - self.v
            if delta_v < 0:
                t_collide = delta_s / delta_v
                if t_collide > 0:
                    F_back = -10 * (float(t_collide) / self.preferred_buffer)**2
                    F_net += F_back
                    if is_ego: print "t_collide, f_back: {}, {}".format(t_collide, F_back)
        
        if is_ego: 
            print "total force on ego is {}".format(F_net)
        if F_net > 0: 
            self.a = int(min(MAX_ACCEL, F_net))
        else:
            self.a = int(max(-MAX_ACCEL, F_net))
            
    def __repr__(self):
        s = "s:    {}\n".format(self.s)
        s +="lane: {}\n".format(self.lane)
        s +="v:    {}\n".format(self.v)
        s +="a:    {}\n".format(self.a)
        return s
        
    def increment(self, dt=1):
        if self.state == "KL":
            self.s = self.s + self.v * dt
            self.v = self.v + self.a * dt
        elif self.state == "LCL":
            self.lane -= 1
            self.v = LANE_SPEEDS[self.lane]
            self.a = 0
            self.increment()
        elif self.state == "LCR":
            pass
        
    def state_at(self, t):
        s = self.s + self.v * t + self.a * t * t / 2
        v = self.v + self.a * t
        return self.lane, s, v, self.a
    
    def collides_with(self, other, at_time=0):
        l,   s,   v,   a = self.state_at(at_time)
        l_o, s_o, v_o, a_o = other.state_at(at_time)
        return l == l_o and abs(s-s_o) <= L 
    
    def will_collide_with(self, other, timesteps):
        for t in range(timesteps+1):
            if self.collides_with(other, t):
                return True, t
        return False, None
        
    def __gt__(self, other):
        return self.s > other.s
    
    def __lt__(self, other):
        return self.s < other.s
    
    def __eq__(self, other):
        return self.s == other.s
    
    def __ge__(self,other):
        return self > other or self == other
    
    def __le__(self,other):
        return self < other or self == other
    
    def __ne__(self, other):
        return not (self == other)
    

In [1234]:
road = Road(SPEED_LIMIT, TRAFFIC_DENSITY, LANE_SPEEDS)
road.populate_traffic()
road.add_ego(2,0)
# road.simulated_update()
for v in road.vehicles.values():
    v.increment()
road.cull()

adding vehicle 32 at lane 0 with s=39
adding vehicle 33 at lane 1 with s=41
adding vehicle 34 at lane 0 with s=30
adding vehicle 35 at lane 0 with s=39
adding vehicle 36 at lane 3 with s=-13
adding vehicle 37 at lane 2 with s=33
adding vehicle 38 at lane 1 with s=41
adding vehicle 39 at lane 0 with s=29
adding vehicle 40 at lane 0 with s=31
adding vehicle 41 at lane 1 with s=30
adding vehicle 42 at lane 0 with s=33


In [1235]:

# for v in road.vehicles.values():
#     v.increment()
road.increment_vehicles()
road.cull()
print road

delta s between -1 and  ***  is 2
|     |     |     |     |
|     |     |     |     |
|     |     |     |     |
|     |     |     |     |
|     |     |     |     |
|     |     |     |     |
| 001 |     |     |     |
|     |     |     |     |
|     |     |     |     |
|     |     |     |     |
|     |     | 014 |     |
|     |     |     | 028 |
| 002 |     |     |     |
|     |     |     |     |
|     |     |     |     |
|     |     |     |     |
|     |     | *** |     |
|     |     |     |     |
|     |     |     |     |
|     |     |     |     |
|     |     |     |     |
|     |     |     | 029 |
|     | 007 |     |     |
| 003 |     |     |     |
|     |     |     |     |
|     |     |     |     |
|     |     |     |     |
|     |     |     |     |
|     | 008 |     | 030 |
|     |     |     |     |
|     |     | 016 |     |
|     |     |     |     |
| 004 |     | 017 |     |
|     |     |     |     |
|     |     | 018 | 031 |
| 039 |     |     |     |
| 034 |     |     |     |
| 04

In [1194]:
road.vehicles[road.ego_key]

s:    185
lane: 2
v:    8
a:    0

In [1173]:
road.vehicles[road.ego_key].v = 10